#### Experimentation du principes de tirages de plus court chemins entre sources et destinations à l'échelle global

Objectif :

- Avant tout, convertir les données du graphe Noe4j utiles dans les bons types (code insee -> Integer, travel_time -> Float)

- Récupérer, à l'échelle du territoire global cette fois, les deux listes sources et destinations, organisées sur la distribution respectives du nombres de véhicules immatriculés (pour les sources) et du score touristique (pour les destinations)

- Expérimenter une preuve de concept de 10 000 tirages aléatoires entre source et destination (pris aléatoirement dans les deux listes), avec requête sous Neo4j et algorithme de plus court chemin.

- Les résultats s'organisent en chemin de noeud. Dans la même requête Neo4j, il faut ensuite transformer ces noeuds de ROAD_POINT en communes les plus proches. La reqête est ainsi décomposée en 2 sous-requêtes.

- Récupérer le résultat final sous forme d'un dataframe (ou d'une liste de liste)

Imports :

In [1]:
import numpy as np

import random

import pandas as pd

import pickle

from neo4j import GraphDatabase, basic_auth

import plotly.express as px

import geopy.distance

In [2]:
PASSWORD_NEO4J = 'passwordneo4j'

In [3]:
driver = GraphDatabase.driver(
  "bolt://localhost:7687",
  auth=basic_auth("neo4j", PASSWORD_NEO4J))

In [4]:
NB_DRAWS = 10000

RATIO_EXCLUDE = 0.3 # proportion des neouds exclus du chemin (pour éviter une surrestimation des grandes villes de départ)

MINIMAL_SUBPATH = 6 # longueur de chemin final minimal (après réduction grâce au ratio précédent) pour que ce chemin soit pris en compte et transmis

---

Cypher Request :

In [1]:
def convert_insee_to_integer(tx):

    query = "MATCH (c:CITY) \
                SET c.insee = ToInteger(c.insee)"
    
    result = tx.run(query)
    return result.data()

In [9]:
def convert_travel_time_to_float(tx):

    query = "MATCH ()-[r:RELATED]-() \
                SET r.travel_time = round(ToFloat(r.travel_time), 2)"
    
    result = tx.run(query)
    return result.data()

In [88]:
def travel_time_to_NEARLY_rel(tx):

    query = "MATCH ()-[r:NEARLY_TO]-() \
                SET r.travel_time = ToFloat(0)"
    
    result = tx.run(query)
    return result.data()

In [5]:
def projection_graph(tx):

    query = "CALL gds.graph.project( \
    'map_shortest_path_5', \
    { \
        CITY: {properties: ['insee', 'x', 'y']}, \
        ROAD_POINT: {properties: ['x', 'y']} \
            }, \
    { \
        NEARLY_TO: {properties:'travel_time', orientation : 'UNDIRECTED'}, \
        RELATED: {properties:'travel_time', orientation : 'NATURAL'} \
        }) \
    YIELD graphName, nodeProjection,  nodeCount, relationshipProjection, relationshipCount \
    RETURN graphName, nodeProjection,  nodeCount, relationshipProjection, relationshipCount"

    result = tx.run(query)
    return result.data()


Algorithme de Dijkstra :

In [137]:
def shortest_path_new(tx, props_list, minimal_duration):

    query = "UNWIND $props_list AS map \
            MATCH (source:CITY {insee: map.source_insee}), (target:CITY {insee: map.target_insee}) \
                CALL gds.shortestPath.dijkstra.stream( \
                'map_shortest_path_5', \
                    { \
                    sourceNode: source, \
                    targetNode: target, \
                    relationshipWeightProperty: 'travel_time' \
                        }) \
            YIELD nodeIds, totalCost, costs \
            CALL apoc.when( \
                totalCost >= " + str(minimal_duration) + ", \
                    ' MATCH (p:ROAD_POINT)-[NEARLY_TO]->(c:CITY) \
                    WHERE ID(p) IN nodeIds \
                    RETURN apoc.coll.toSet(collect(c.insee)) AS col', \
                    'RETURN NULL AS col', \
                    {nodeIds : nodeIds} \
                    ) \
                    YIELD value \
                RETURN map.source_insee AS source, map.target_insee AS target, value.col AS collection, totalCost, costs"
    
    
    result = tx.run(query, props_list = props_list)
    return result.data()

In [92]:
def shortest_path(tx, props_list):

    query = "UNWIND $props_list AS map \
            MATCH (source:CITY {insee: map.source_insee}), (target:CITY {insee: map.target_insee}) \
                CALL gds.shortestPath.dijkstra.stream( \
                'map_shortest_path_5', \
                    { \
                    sourceNode: source, \
                    targetNode: target, \
                    relationshipWeightProperty: 'travel_time' \
                        }) \
            YIELD nodeIds, totalCost, costs \
            CALL { \
                    WITH nodeIds \
                    MATCH (p:ROAD_POINT)-[NEARLY_TO]->(c:CITY) \
                    WHERE ID(p) IN nodeIds \
                    RETURN apoc.coll.toSet(collect(c.insee)) AS collection \
                }\
                RETURN map.source_insee AS source, map.target_insee AS target, collection, totalCost, costs"
    
    
    result = tx.run(query, props_list = props_list)
    return result.data()

    #WITH nodeIds[ToInteger(size(nodeIds)*"+ str(ratio) + ")..] AS path_list \


Autre essai d'alogorithme  (A*) (attention: en réalité, trois fois plus long !) :

In [25]:
""" def shortest_path_A(tx, props_list):

    query = "UNWIND $props_list AS map \
            MATCH (source:CITY {insee: map.source_insee}), (target:CITY {insee: map.target_insee}) \
                CALL gds.shortestPath.astar.stream( \
                'map_shortest_path_4', \
                    { \
                    sourceNode: source, \
                    targetNode: target, \
                    latitudeProperty: 'y', \
                    longitudeProperty: 'x', \
                    relationshipWeightProperty: 'travel_time' \
                        }) \
            YIELD nodeIds, \
            CALL { \
                        WITH nodeIds \
                        MATCH (p:ROAD_POINT)-[NEARLY_TO]->(c:CITY) \
                        WHERE ID(p) IN nodeIds \
                        RETURN apoc.coll.toSet(collect(c.insee)) AS collection \
                }\
                RETURN collection"
    
    
    result = tx.run(query, props_list = props_list)
    return result.data() """

In [7]:
def get_communes_nearliest(tx, props_list):

    query = "UNWIND $props_list AS map \
                CALL { \
                        WITH map \
                        MATCH (p:ROAD_POINT)-[NEARLY_TO]->(c:CITY) \
                        WHERE ID(p) IN map.nodeIds \
                        RETURN apoc.coll.toSet(collect(c.insee)) AS collection \
                } \
                RETURN collection"
    
    result = tx.run(query, props_list = props_list)
    return result.data()

In [8]:
def get_all_communes_with_coords(tx):

    query = "MATCH (n:CITY) RETURN n.insee as insee, n.x as x, n.y as y"

    result = tx.run(query)
    return result.data()


---

Requêtes de preprocessing :

Conversion des codes insee en entiers (pour accélérer les calculs)

In [228]:
with driver.session() as session:

    result = session.execute_write(convert_insee_to_integer)

driver.close()

print(result)

[]


Conversion des "travel_time" des routes en float (car les "travel_time" vont servir de poids aux algorithmes de plus court chemins)

In [229]:
with driver.session() as session:

    result = session.execute_write(convert_travel_time_to_float)

driver.close()

print(result)

[]


Ajout d'un "travel_time" par défaut à zéro pour les relations "NEARLY_TO" entre CITY et ROAD POINT pour prévenir les bugs

In [89]:
with driver.session() as session:

    result = session.execute_write(travel_time_to_NEARLY_rel)

driver.close()

print(result)

[]


---

Requête de projection (pour emploi de l'algo de plus court chemin)

In [9]:
with driver.session() as session:

    count_list = []

    result = session.execute_write(projection_graph)

driver.close()

print(result)

[{'graphName': 'map_shortest_path_5', 'nodeProjection': {'ROAD_POINT': {'label': 'ROAD_POINT', 'properties': {'x': {'defaultValue': None, 'property': 'x'}, 'y': {'defaultValue': None, 'property': 'y'}}}, 'CITY': {'label': 'CITY', 'properties': {'insee': {'defaultValue': None, 'property': 'insee'}, 'x': {'defaultValue': None, 'property': 'x'}, 'y': {'defaultValue': None, 'property': 'y'}}}}, 'nodeCount': 557768, 'relationshipProjection': {'RELATED': {'orientation': 'NATURAL', 'indexInverse': False, 'aggregation': 'DEFAULT', 'type': 'RELATED', 'properties': {'travel_time': {'defaultValue': None, 'property': 'travel_time', 'aggregation': 'DEFAULT'}}}, 'NEARLY_TO': {'orientation': 'UNDIRECTED', 'indexInverse': False, 'aggregation': 'DEFAULT', 'type': 'NEARLY_TO', 'properties': {'travel_time': {'defaultValue': None, 'property': 'travel_time', 'aggregation': 'DEFAULT'}}}}, 'relationshipCount': 1359891}]


----

Récupération des listes aléatoires de sources et destinations :

In [10]:
with open('./datas/source_list_test.pkl', 'rb') as f:
    source_list = pickle.load(f)

with open('./datas/target_list_test.pkl', 'rb') as f:
    target_list = pickle.load(f)

Shuffling (selon nombre de tirages)

In [11]:
shuffle_source = [random.choice(source_list) for i in range(NB_DRAWS)]
shuffle_target = [random.choice(target_list) for i in range(NB_DRAWS)]

Conversion en dataframe :

In [12]:
df_shuffle = pd.DataFrame({"source_insee" : shuffle_source, "target_insee" : shuffle_target})

In [99]:
# df_shuffle_test = df_shuffle.iloc[0:10,:]

In [152]:
props_list = df_shuffle.to_dict('records')[0:10]

In [153]:
props_list

[{'source_insee': 34057, 'target_insee': 1283},
 {'source_insee': 87048, 'target_insee': 67306},
 {'source_insee': 38389, 'target_insee': 66065},
 {'source_insee': 6152, 'target_insee': 66004},
 {'source_insee': 31303, 'target_insee': 74139},
 {'source_insee': 62397, 'target_insee': 68335},
 {'source_insee': 60172, 'target_insee': 62472},
 {'source_insee': 13001, 'target_insee': 75056},
 {'source_insee': 94065, 'target_insee': 76656},
 {'source_insee': 69275, 'target_insee': 69253}]

In [154]:
with driver.session() as session:

    result = session.execute_write(shortest_path_new, props_list, MINIMAL_TIME)

    #print(result[0])

    #result = session.execute_write(get_communes_nearliest, result)

driver.close()

In [155]:
paths = pd.DataFrame(result)

In [156]:
paths

,source,target,collection,totalCost,costs
0,34057,1283,"[34057, 34327, 30116, 30221, 84027, 7022, 2634...",13953.5,"[0.0, 0.0, 26.0, 27.6, 27.700000000000003, 30...."
1,87048,67306,"[87048, 87014, 23002, 23149, 71590, 71040, 711...",23354.9,"[0.0, 0.0, 40.8, 41.099999999999994, 45.599999..."
2,38389,66065,"[38389, 38288, 38480, 38484, 38544, 69189, 383...",17542.1,"[0.0, 0.0, 403.5, 403.5, 403.5, 405.8, 411.1, ..."
3,6152,66004,"[6152, 6007, 83133, 83099, 13025, 30258, 30341...",20629.7,"[0.0, 0.0, 143.8, 145.3, 145.70000000000002, 1..."
4,31303,74139,"[31303, 31576, 11430, 11438, 11106, 34183, 301...",22723.2,"[0.0, 0.0, 57.4, 58.199999999999996, 63.199999..."
5,62397,68335,"[62397, 62031, 62203, 62174, 62078, 62038, 626...",20360.9,"[0.0, 0.0, 45.3, 46.099999999999994, 47.8, 49...."
6,60172,62472,"[60172, 95149, 95352, 95026, 95456, 60398, 600...",7341.3,"[0.0, 0.0, 101.8, 103.7, 104.3, 105.3999999999..."
7,13001,75056,"[13001, 7022, 26347, 26380, 26119, 26247, 2633...",26290.9,"[0.0, 0.0, 62.6, 66.3, 78.89999999999999, 92.3..."
8,94065,76656,None,5948.5,"[0.0, 0.0, 30.3, 36.3, 37.9, 38.69999999999999..."
9,69275,69253,None,2326.9,"[0.0, 0.0, 2.9, 4.0, 6.0, 23.3, 29.5, 64.9, 66..."


In [176]:
with driver.session() as session:

    result = session.execute_read(shortest_path, props_list)

    #print(result[0])

    #result = session.execute_write(get_communes_nearliest, result)

driver.close()

In [179]:
type(result)

list

In [158]:
paths = pd.DataFrame(result)

In [159]:
paths

,source,target,collection,totalCost,costs
0,34057,1283,"[34057, 34327, 30116, 30221, 84027, 7022, 2634...",13953.5,"[0.0, 0.0, 26.0, 27.6, 27.700000000000003, 30...."
1,87048,67306,"[87048, 87014, 23002, 23149, 71590, 71040, 711...",23354.9,"[0.0, 0.0, 40.8, 41.099999999999994, 45.599999..."
2,38389,66065,"[38389, 38288, 38480, 38484, 38544, 69189, 383...",17542.1,"[0.0, 0.0, 403.5, 403.5, 403.5, 405.8, 411.1, ..."
3,6152,66004,"[6152, 6007, 83133, 83099, 13025, 30258, 30341...",20629.7,"[0.0, 0.0, 143.8, 145.3, 145.70000000000002, 1..."
4,31303,74139,"[31303, 31576, 11430, 11438, 11106, 34183, 301...",22723.2,"[0.0, 0.0, 57.4, 58.199999999999996, 63.199999..."
5,62397,68335,"[62397, 62031, 62203, 62174, 62078, 62038, 626...",20360.9,"[0.0, 0.0, 45.3, 46.099999999999994, 47.8, 49...."
6,60172,62472,"[60172, 95149, 95352, 95026, 95456, 60398, 600...",7341.3,"[0.0, 0.0, 101.8, 103.7, 104.3, 105.3999999999..."
7,13001,75056,"[13001, 7022, 26347, 26380, 26119, 26247, 2633...",26290.9,"[0.0, 0.0, 62.6, 66.3, 78.89999999999999, 92.3..."
8,94065,76656,"[94065, 94021, 94003, 92049, 92046, 92075, 920...",5948.5,"[0.0, 0.0, 30.3, 36.3, 37.9, 38.69999999999999..."
9,69275,69253,"[69275, 69277, 69287, 69298, 69283, 38487, 385...",2326.9,"[0.0, 0.0, 2.9, 4.0, 6.0, 23.3, 29.5, 64.9, 66..."


In [34]:
"""paths["length_collection"] = paths['collection'].apply(lambda list : len(list))
paths["length_costs"] = paths['costs'].apply(lambda list : len(list)) """

In [78]:
MINIMAL_TIME = 7200 # 2 heures de routes soit en moyenne 120-200 km (autonomie usuelle sans recharge)

In [160]:
paths = paths.loc[paths["totalCost"] >= MINIMAL_TIME,:]

In [161]:
paths

,source,target,collection,totalCost,costs
0,34057,1283,"[34057, 34327, 30116, 30221, 84027, 7022, 2634...",13953.5,"[0.0, 0.0, 26.0, 27.6, 27.700000000000003, 30...."
1,87048,67306,"[87048, 87014, 23002, 23149, 71590, 71040, 711...",23354.9,"[0.0, 0.0, 40.8, 41.099999999999994, 45.599999..."
2,38389,66065,"[38389, 38288, 38480, 38484, 38544, 69189, 383...",17542.1,"[0.0, 0.0, 403.5, 403.5, 403.5, 405.8, 411.1, ..."
3,6152,66004,"[6152, 6007, 83133, 83099, 13025, 30258, 30341...",20629.7,"[0.0, 0.0, 143.8, 145.3, 145.70000000000002, 1..."
4,31303,74139,"[31303, 31576, 11430, 11438, 11106, 34183, 301...",22723.2,"[0.0, 0.0, 57.4, 58.199999999999996, 63.199999..."
5,62397,68335,"[62397, 62031, 62203, 62174, 62078, 62038, 626...",20360.9,"[0.0, 0.0, 45.3, 46.099999999999994, 47.8, 49...."
6,60172,62472,"[60172, 95149, 95352, 95026, 95456, 60398, 600...",7341.3,"[0.0, 0.0, 101.8, 103.7, 104.3, 105.3999999999..."
7,13001,75056,"[13001, 7022, 26347, 26380, 26119, 26247, 2633...",26290.9,"[0.0, 0.0, 62.6, 66.3, 78.89999999999999, 92.3..."


In [ ]:
vis

In [81]:
# Binary Search in python


def binarySearch(array, x, low, high):

    if high >= low:

        mid = low + (high - low)//2

        # If found at mid, then return it
        if array[mid] == x:
            return mid

        # Search the left half
        elif array[mid] > x:
            return binarySearch(array, x, low, mid-1)

        # Search the right half
        else:
            return binarySearch(array, x, mid + 1, high)

    else:
        return low

In [83]:
def filter_begining(df, minimal_duration):

    costs_list = df["costs"].tolist()
    collection_list = df["collection"].tolist()

    new_collection = []

    df_len = len(df)

    for i in range(df_len):

        communes_array = collection_list[i]

        len_communes_array = len(communes_array)

        costs_array = costs_list[i]

        len_costs_array = len(costs_array)

        index_min = binarySearch(costs_array, minimal_duration, 0, len_costs_array - 1)

        ratio = index_min / len_costs_array

        index_begin = int(ratio*len_communes_array)

        new_collection.append(communes_array[index_begin:])

    df["collection"] = new_collection

    df.drop(["costs"], axis=1, inplace=True)

    return df


In [172]:
paths = filter_begining(paths, MINIMAL_TIME)

/tmp/ipykernel_5433/309902433.py:28: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_5433/309902433.py:30: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [173]:
paths

,source,target,collection,totalCost
0,34057,1283,"[26333, 26160, 38487, 69295, 69298, 69287, 692...",13953.5
1,87048,67306,"[21512, 21412, 21712, 21037, 21428, 52519, 521...",23354.9
2,38389,66065,"[26247, 26119, 26380, 26347, 26339, 26235, 840...",17542.1
3,6152,66004,"[34183, 11106, 11024, 66174, 66230, 66007, 661...",20629.7
4,31303,74139,"[7022, 38368, 38331, 38105, 38222, 38386, 3837...",22723.2
5,62397,68335,"[2785, 2382, 2330, 2756, 2282, 2218, 2229, 247...",20360.9
6,60172,62472,"[62698, 62472]",7341.3
7,13001,75056,"[1430, 1151, 1355, 1343, 1332, 1320, 71448, 71...",26290.9


In [133]:
paths.to_parquet("./datas/path_sample_10000_new.parquet")

Re-import :

In [38]:
"""test_df = pd.read_parquet("./datas/test_paths_mini.parquet")"""

----

Visualisations de test

Réupération des coordonnées des communes :

In [150]:
with driver.session() as session:

    result = session.execute_write(get_all_communes_with_coords)

driver.close()

df_communes = pd.DataFrame(result)

In [169]:
def visualize_one_path(df, index):

    index_list = df.iloc[index,2]

    print(index_list)

    print(f" Path : {index_list}")

    df_reduce = df_communes.loc[df_communes['insee'].isin(index_list),:]

    fig = px.scatter_mapbox(df_reduce, lat='y', lon='x', hover_name='insee', mapbox_style='open-street-map')

    fig.show()

    

In [175]:
paths

,source,target,collection,totalCost
0,34057,1283,"[26333, 26160, 38487, 69295, 69298, 69287, 692...",13953.5
1,87048,67306,"[21512, 21412, 21712, 21037, 21428, 52519, 521...",23354.9
2,38389,66065,"[26247, 26119, 26380, 26347, 26339, 26235, 840...",17542.1
3,6152,66004,"[34183, 11106, 11024, 66174, 66230, 66007, 661...",20629.7
4,31303,74139,"[7022, 38368, 38331, 38105, 38222, 38386, 3837...",22723.2
5,62397,68335,"[2785, 2382, 2330, 2756, 2282, 2218, 2229, 247...",20360.9
6,60172,62472,"[62698, 62472]",7341.3
7,13001,75056,"[1430, 1151, 1355, 1343, 1332, 1320, 71448, 71...",26290.9


In [174]:
visualize_one_path(paths, 3)

[34183, 11106, 11024, 66174, 66230, 66007, 66103, 66161, 66054, 66223, 66193, 66125, 66010, 66047, 66157, 66105, 66004]
 Path : [34183, 11106, 11024, 66174, 66230, 66007, 66103, 66161, 66054, 66223, 66193, 66125, 66010, 66047, 66157, 66105, 66004]
